# Preview Local .tif Satellite Images

Use this notebook to:

- List `.tif` files in `data/sentinel2/`
- Inspect and export per-file metadata
- Preview images inline (RGB composite if at least 3 bands, otherwise single-band)
- Save preview thumbnails and export metadata to Excel

Notes:
- Ensure `rasterio`, `matplotlib`, and `ipywidgets` are installed in your environment.
- Run cells in order.

In [ ]:
# Prerequisites (uncomment to install in the notebook environment)
# !pip install rasterio matplotlib ipywidgets openpyxl

# Imports
import os
import glob
import pandas as pd
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
import ipywidgets as widgets

# Configure inline plotting
%matplotlib inline

In [ ]:
# List .tif files in data/sentinel2/
TIF_DIR = os.path.join("data", "sentinel2")
tif_paths = sorted(glob.glob(os.path.join(TIF_DIR, "*.tif")))

if not tif_paths:
    print(f"No .tif files found in '{TIF_DIR}'.\nIf your files live elsewhere, update `TIF_DIR` to the correct path.")
else:
    print(f"Found {len(tif_paths)} .tif file(s) in: {TIF_DIR}")
    
    # Build metadata DataFrame
    meta_list = []
    for p in tif_paths:
        try:
            with rasterio.open(p) as src:
                meta_list.append({
                    "path": p,
                    "filename": os.path.basename(p),
                    "width": src.width,
                    "height": src.height,
                    "bands": src.count,
                    "crs": str(src.crs),
                    "bounds": tuple(src.bounds),
                    "dtype": src.dtypes[0],
                    "driver": src.driver,
                })
        except Exception as e:
            meta_list.append({"path": p, "filename": os.path.basename(p), "error": str(e)})

    df_meta = pd.DataFrame(meta_list)
    display(df_meta.head())
    print(f"Metadata table has {len(df_meta)} rows.")

## Select and Preview a .tif File

Use the dropdown below to pick a file. The notebook will print metadata and show an inline preview. A small thumbnail preview PNG will also be written to `previews/`.

In [ ]:
# Widget for file selection and preview
if tif_paths:
    options = [(os.path.basename(p), p) for p in tif_paths]
    file_dropdown = widgets.Dropdown(options=options, description='File:')
    display(file_dropdown)
    output = widgets.Output()
    display(output)

    PREVIEW_DIR = "previews"
    os.makedirs(PREVIEW_DIR, exist_ok=True)

    def _normalize_band(b):
        b = b.astype('float32')
        b = b - b.min()
        if b.max() > 0:
            b = b / b.max()
        return b

    def preview_file(path):
        output.clear_output()
        with output:
            try:
                with rasterio.open(path) as src:
                    print(f"Path: {path}")
                    print(f"Size: {src.width}x{src.height}, Bands: {src.count}")
                    print(f"CRS: {src.crs}")
                    print(f"Bounds: {src.bounds}")
                    print(f"Dtype: {src.dtypes}")

                    # RGB composite if at least 3 bands
                    if src.count >= 3:
                        r = src.read(1)
                        g = src.read(2)
                        b = src.read(3)
                        rgb = np.dstack([_normalize_band(r), _normalize_band(g), _normalize_band(b)])
                        fig, ax = plt.subplots(figsize=(8, 8))
                        ax.imshow(rgb)
                        ax.axis('off')
                        plt.show()

                        # Save preview thumbnail
                        thumb_path = os.path.join(PREVIEW_DIR, os.path.basename(path) + '.png')
                        plt.imsave(thumb_path, rgb)
                        print(f"Saved preview thumbnail: {thumb_path}")

                    else:
                        band1 = src.read(1)
                        fig, ax = plt.subplots(figsize=(8, 8))
                        ax.imshow(band1, cmap='gray')
                        ax.axis('off')
                        plt.colorbar(ax.images[0], ax=ax, fraction=0.046, pad=0.04)
                        plt.show()

            except Exception as e:
                print(f"Error reading file: {e}")

    def _on_change(change):
        if change['name'] == 'value' and change['new']:
            preview_file(change['new'])

    file_dropdown.observe(_on_change)

## Export Metadata to Excel

Save the collected metadata to an Excel file for offline inspection.

In [ ]:
if 'df_meta' in globals() and not df_meta.empty:
    excel_path = 'satellite_images_metadata.xlsx'
    df_meta.to_excel(excel_path, index=False)
    print(f"Exported metadata to: {excel_path}")
else:
    print('No metadata to export.')